# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study11'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 100-200 --> 205:
	{'snapshot_id': 'AAABnVQevEY8qp9trSaQKbpiLuoVWptF'}
Iteration 2, 100-200 --> 343:
	{'snapshot_id': 'AAABnrFOyIRlKBu4C85oR9RedZwu+mEI'}
Iteration 3, 0-100 --> 338:
	{'snapshot_id': 'AAABn17O23nlh9+f2/XRkfNqgrRt/Bds'}

Phase 2: Chunks of 10

Iteration 1, 190-200 --> 14:
	{'snapshot_id': 'AAABoBSrpQ2/puhVXUPTccjdp0ObY9WR'}
Iteration 2, 240-250 --> 263:
	{'snapshot_id': 'AAABod4OnZ0cTAWewwdQAdsh8pUNRiWn'}
Iteration 3, 360-370 --> 307:
	{'snapshot_id': 'AAABotYDLvYLI+NCux/HxRiifvepgzU/'}
Iteration 4, 260-270 --> 155:
	{'snapshot_id': 'AAABo2GBEaA9a7JBR0jQPxcbYtCR50xe'}
Iteration 5, 130-140 --> 282:
	{'snapshot_id': 'AAABpMDWedZ+bs3VwQaY1ovWqJRR318a'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/65MXR4dubPL9t0P4dgTWvn/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 6, 320-330 --> 283:
	{'snapshot_id': 'AAABpegFJgnmbcFUsfDk+pQzw0vMY48y'}
ERROR, 380-380 --> 336:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/65MXR4dubPL9t0P4dgTWvn/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 8, 220-230 --> 297:
	{'snapshot_id': 'AAABpkR1OEJJmST6yOW+G3oggx49L0+k'}
Iteration 9, 230-240 --> 306:
	{'snapshot_id': 'AAABpzShfeiRNqP1em131OGCsLTpN65L'}
Iteration 10, 140-150 --> 215:
	{'snapshot_id': 'AAABqEg5vamAI/FlfqSTf7k8GoEIoXeM'}
Iteration 11, 330-340 --> 207:
	{'snapshot_id': 'AAABqTD8yuSz1LGfj+fUjnE6+IOM9yU3'}
Iteration 12, 10-20 --> 89:
	{'snapshot_id': 'AAABqjgNGzCUt7hyPpXpfv4CWRW8MVCq'}
Iteration 13, 220-230 --> 60:
	{'snapshot_id': 'AAABq4CYZptlkiA1mT3dfx9rkWGly7j0'}
Iteration 14, 230-240 --> 62:
	{'snapshot_id': 'AAABrBTGKnDYaTEl3D1343/4NXTYXsqT'}
Iteration 15, 290-300 --> 101:
	{'snapshot_id': 'AAABrTEpnZAxsATvlBDng2BI1ZvasTSa'}
Iteration 16, 150-160 --> 319:
	{'snapshot_id': 'AAABrjc6oYYDgteyH

HTTP Error for PUT to https://api.spotify.com/v1/playlists/65MXR4dubPL9t0P4dgTWvn/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 21, 350-360 --> 43:
	{'snapshot_id': 'AAABs0aRPjgNOL/AtdaWMWU2naUsY4OT'}
ERROR, 380-380 --> 281:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/65MXR4dubPL9t0P4dgTWvn/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 23, 310-320 --> 202:
	{'snapshot_id': 'AAABtNCn5Y3gZC8H07u3/N2rmRKRaXDf'}
Iteration 24, 90-100 --> 103:
	{'snapshot_id': 'AAABtVySqAXP6S7XaesP7iAaAr7EUlZz'}
Iteration 25, 130-140 --> 170:
	{'snapshot_id': 'AAABtkJ2NejXCb/cNnoIxz40lAe7SxoJ'}
Iteration 26, 340-350 --> 175:
	{'snapshot_id': 'AAABt1AGE/yhd/LZ45hsIqATkKcuoW+C'}
Iteration 27, 310-320 --> 239:
	{'snapshot_id': 'AAABuG8q1qK2/kUs68dUNxTShbhS9fyy'}
Iteration 28, 120-130 --> 273:
	{'snapshot_id': 'AAABuc9K9xuy6fYsA13T8td3pD4bLqFF'}
Iteration 29, 130-140 --> 230:
	{'snapshot_id': 'AAABurwvqLffHvib1Q7Tw1cUPCwpaYem'}
Iteration 30, 50-60 --> 157:
	{'snapshot_id': 'AAABu+y4lyKnz6g4/VIz+KfR8pUbo25y'}
Iteration 31, 270-280 --> 140:
	{'snapshot_id': 'AAABvOgdU1pH+